# Main Results - Dispatch

Developed by Siobhan Powell, 2021. 


In [1]:
import os
os.chdir('..')

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import scipy
import os.path
import pandas
import matplotlib
import shutil
import copy

from simple_dispatch import generatorData
from simple_dispatch import bidStack
from simple_dispatch import dispatch
from simple_dispatch import generatorDataShort
from future_grid import FutureDemand
from future_grid import FutureGrid
from simple_dispatch import StorageModel

# Load Generator Model

In [4]:
gd_short = pickle.load(open('IntermediateOutputs/generator_data_short_WECC_2019.obj', 'rb'))

# Dispatch Main Grid Scenario 

Wind and Solar 2.5X 2019 levels

In [4]:
max_rate = 10000 # assumes average 4 hour storage 
cap = 40000

# Load capacity result so know if need to cover with storage
maxpen_results2 = pd.read_csv('Results/max_penetration_levels_1h_20220330.csv', index_col=0)
maxpen_results2_bau = pd.read_csv('Results/max_penetration_levels_1h_bau_20220330.csv', index_col=0)

timer_names = {'':'Timers9pm', '_midnighttimer':'Timers12am', '_NoTimers':'TimersNone', '_RandomTimers':'TimersRandom'}

fuel = 1
ev_scenario_date = '20220313'

for penlevel in [0.5, 1.0]:
    for solar, wind in {2.5:2.5}.items(): # Medium renewables scenario
        for ev_scenario in ['UniversalHome', 'HighHome', 'LowHome_HighWork', 'LowHome_LowWork']: # Access scenarios
            # no workplace control
            ev_workplace_bool=False
            for i, ev_timers in enumerate(['_RandomTimers', '', '_midnighttimer', '_NoTimers']):
                # Case without storage before
                grid = FutureGrid(gd_short)
                grid.set_up_scenario(year=2030, solar=solar, wind=wind, fuel=fuel, ev_scenario=ev_scenario, ev_timers=ev_timers, ev_pen=penlevel, 
                                     ev_workplace_control='', ev_workplace_bool=False, evs_bool=True, ev_scenario_date=ev_scenario_date, 
                                     weekend_date=ev_scenario_date,  weekend_timers=ev_timers)
                save_str = 'Results/Fuel1_Solar25_Wind25/fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_'+timer_names[ev_timers]+'_noWPcontrol'+'_penlevel'+str(penlevel)
                print(save_str)
                str1 = 'fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_noWPcontrol'
                max_penlevel = maxpen_results2.loc[timer_names[ev_timers], str1]
                grid.check_overgeneration(save_str=save_str)
                grid.run_dispatch(max_penlevel, save_str, result_date='20220330') 

                # Case with 10GW storage before
                grid = FutureGrid(gd_short)
                grid.set_up_scenario(year=2030, solar=solar, wind=wind, fuel=fuel, ev_scenario=ev_scenario, 
                                     ev_timers=ev_timers, ev_pen=penlevel, ev_workplace_control='', ev_workplace_bool=False, evs_bool=True, 
                                     ev_scenario_date=ev_scenario_date, weekend_date=ev_scenario_date,  weekend_timers=ev_timers)
                save_str = 'Results/Fuel1_Solar25_Wind25/fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_'+timer_names[ev_timers]+'_noWPcontrol'+'_penlevel'+str(penlevel)+'_storagebefore'
                print(save_str)
                str1 = 'fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_noWPcontrol'
                grid.check_overgeneration(save_str=save_str, change_demand=False)
                grid.run_storage_before_capacitydispatch(cap, max_rate, allow_negative=True)
                grid.storage.df.to_csv(save_str+'_storagebeforedf_'+'20220330'+'.csv')
                # Change demand dispatched to fossil fuel generators based on storage operations:
                grid.future.demand['demand'] = np.copy(grid.storage.df.comb_demand_after_storage.values)
                grid.check_overgeneration(save_str=save_str, change_demand=True, extra_save_str='_afterstorage')
                if penlevel < 0.8:
                    max_penlevel = 1.0
                grid.run_dispatch(max_penlevel, save_str, result_date='20220330') 

            # workplace control - repeat above without timers but with workplace controls
            ev_workplace_bool=True
            ev_timers = '_NoTimers'
            for wp_control in ['avgem', 'minpeak']:
                # Case without storage
                grid = FutureGrid(gd_short)
                grid.set_up_scenario(year=2030, solar=solar, wind=wind, fuel=fuel, ev_scenario=ev_scenario, 
                                                        ev_timers=ev_timers, ev_pen=penlevel, ev_workplace_control=wp_control, ev_workplace_bool=True, evs_bool=True, 
                                                        ev_scenario_date=ev_scenario_date, weekend_date=ev_scenario_date,  weekend_timers=ev_timers)
                save_str = 'Results/Fuel1_Solar25_Wind25/fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_'+timer_names[ev_timers]+'_WPcontrol_'+wp_control+'_penlevel'+str(penlevel)
                print(save_str)
                str1 = 'fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_WPcontrol_'+wp_control
                max_penlevel = maxpen_results2.loc[timer_names[ev_timers], str1]
                grid.check_overgeneration(save_str=save_str)
                grid.run_dispatch(max_penlevel, save_str, result_date='20220330')

                # Case with storage before
                grid = FutureGrid(gd_short)
                grid.set_up_scenario(year=2030, solar=solar, wind=wind, fuel=fuel, ev_scenario=ev_scenario, 
                                                        ev_timers=ev_timers, ev_pen=penlevel, ev_workplace_control=wp_control, ev_workplace_bool=True, evs_bool=True, 
                                                        ev_scenario_date=ev_scenario_date, weekend_date=ev_scenario_date,  weekend_timers=ev_timers)

                save_str = 'Results/Fuel1_Solar25_Wind25/fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_'+timer_names[ev_timers]+'_WPcontrol_'+wp_control+'_penlevel'+str(penlevel)+'_storagebefore'
                str1 = 'fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_WPcontrol_'+wp_control
                print(save_str)
                grid.check_overgeneration(save_str=save_str, change_demand=False)
                grid.run_storage_before_capacitydispatch(cap, max_rate, allow_negative=True)
                grid.storage.df.to_csv(save_str+'_storagebeforedf_'+'20220330'+'.csv')
                grid.future.demand['demand'] = np.copy(grid.storage.df.comb_demand_after_storage.values)
                grid.check_overgeneration(save_str=save_str, change_demand=True, extra_save_str='_afterstorage')

                if penlevel < 0.8:
                    max_penlevel = 1.0
                grid.run_dispatch(max_penlevel, save_str, result_date='20220330') 
                
                

        ev_scenario = 'BusinessAsUsual'
        ev_workplace_bool=True
        ev_timers=''
        wp_control = 'minpeak'
        grid = FutureGrid(gd_short)
        grid.set_up_scenario(year=2030, solar=solar, wind=wind, fuel=fuel, ev_scenario=ev_scenario, 
                                                    ev_timers=ev_timers, ev_pen=penlevel, ev_workplace_control=wp_control, ev_workplace_bool=True, evs_bool=True, 
                                                    ev_scenario_date=ev_scenario_date, weekend_date=ev_scenario_date,  weekend_timers=ev_timers)
        save_str = 'Results/Fuel1_Solar25_Wind25/fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_'+'TimersMixed'+'_WPcontrol_'+wp_control+'_penlevel'+str(penlevel)
        str1 = 'fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_WPcontrol_'+wp_control
        max_penlevel = maxpen_results2_bau.loc['TimersMixed', str1]
        print(save_str)
        grid.check_overgeneration(save_str=save_str)
        grid.run_dispatch(max_penlevel, save_str, result_date='20220330')                
            
        # storage before
        grid = FutureGrid(gd_short)
        grid.set_up_scenario(year=2030, solar=solar, wind=wind, fuel=fuel, ev_scenario=ev_scenario, 
                                                    ev_timers=ev_timers, ev_pen=penlevel, ev_workplace_control=wp_control, ev_workplace_bool=True, evs_bool=True, 
                                                    ev_scenario_date=ev_scenario_date, weekend_date=ev_scenario_date,  weekend_timers=ev_timers)
        save_str = 'Results/Fuel1_Solar25_Wind25/fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_'+'TimersMixed'+'_WPcontrol_'+wp_control+'_penlevel'+str(penlevel)+'_storagebefore'
        str1 = 'fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_WPcontrol_'+wp_control
        print(save_str)
        grid.check_overgeneration(save_str=save_str, change_demand=False)
        grid.run_storage_before_capacitydispatch(cap, max_rate, allow_negative=True)
        grid.storage.df.to_csv(save_str+'_storagebeforedf_'+'20220330'+'.csv')
        grid.future.demand['demand'] = np.copy(grid.storage.df.comb_demand_after_storage.values)
        grid.check_overgeneration(save_str=save_str, change_demand=True, extra_save_str='_afterstorage')
        if penlevel < 0.8:
            max_penlevel = 1.0
        grid.run_dispatch(max_penlevel, save_str, result_date='20220330')  
                    
                    

ERROR! Session/line number was not unique in database. History logging moved to new session 2409
Results/Fuel1_Solar25_Wind25/fuel1_solar2.5_wind2.5_UniversalHome_TimersRandom_noWPcontrol_penlevel0.5
1.9% Complete
3.8% Complete
5.8% Complete
7.7% Complete
9.6% Complete
11.5% Complete
13.5% Complete
15.4% Complete
17.3% Complete
19.2% Complete
21.2% Complete
23.1% Complete
25.0% Complete
26.9% Complete
28.8% Complete
30.8% Complete
32.7% Complete
34.6% Complete
36.5% Complete
38.5% Complete
40.4% Complete
42.3% Complete
44.2% Complete
46.2% Complete
48.1% Complete
50.0% Complete
51.9% Complete
53.8% Complete
55.8% Complete
57.7% Complete
59.6% Complete
61.5% Complete
63.5% Complete
65.4% Complete
67.3% Complete
69.2% Complete
71.2% Complete
73.1% Complete
75.0% Complete
76.9% Complete
78.8% Complete
80.8% Complete
82.7% Complete
84.6% Complete
86.5% Complete
88.5% Complete
90.4% Complete
92.3% Complete
94.2% Complete
96.2% Complete
98.1% Complete
100.0% Complete
Results/Fuel1_Solar25_Wi

75.0% Complete
76.9% Complete
78.8% Complete
80.8% Complete
82.7% Complete
84.6% Complete
86.5% Complete
88.5% Complete
90.4% Complete
92.3% Complete
94.2% Complete
96.2% Complete
98.1% Complete
100.0% Complete
Solving optimization.
Storage Rate Result: 751
Storage Capacity:  3005
Results/Fuel1_Solar25_Wind25/fuel1_solar2.5_wind2.5_UniversalHome_TimersNone_WPcontrol_avgem_penlevel0.5_storagebefore
Solving optimization.
1.9% Complete
3.8% Complete
5.8% Complete
7.7% Complete
9.6% Complete
11.5% Complete
13.5% Complete
15.4% Complete
17.3% Complete
19.2% Complete
21.2% Complete
23.1% Complete
25.0% Complete
26.9% Complete
28.8% Complete
30.8% Complete
32.7% Complete
34.6% Complete
36.5% Complete
38.5% Complete
40.4% Complete
42.3% Complete
44.2% Complete
46.2% Complete
48.1% Complete
50.0% Complete
51.9% Complete
53.8% Complete
55.8% Complete
57.7% Complete
59.6% Complete
61.5% Complete
63.5% Complete
65.4% Complete
67.3% Complete
69.2% Complete
71.2% Complete
73.1% Complete
75.0% Comple

69.2% Complete
71.2% Complete
73.1% Complete
75.0% Complete
76.9% Complete
78.8% Complete
80.8% Complete
82.7% Complete
84.6% Complete
86.5% Complete
88.5% Complete
90.4% Complete
92.3% Complete
94.2% Complete
96.2% Complete
98.1% Complete
100.0% Complete
Results/Fuel1_Solar25_Wind25/fuel1_solar2.5_wind2.5_HighHome_TimersNone_noWPcontrol_penlevel0.5
1.9% Complete
3.8% Complete
5.8% Complete
7.7% Complete
9.6% Complete
11.5% Complete
13.5% Complete
15.4% Complete
17.3% Complete
19.2% Complete
21.2% Complete
23.1% Complete
25.0% Complete
26.9% Complete
28.8% Complete
30.8% Complete
32.7% Complete
34.6% Complete
36.5% Complete
38.5% Complete
40.4% Complete
42.3% Complete
44.2% Complete
46.2% Complete
48.1% Complete
50.0% Complete
51.9% Complete
53.8% Complete
55.8% Complete
57.7% Complete
59.6% Complete
61.5% Complete
63.5% Complete
65.4% Complete
67.3% Complete
69.2% Complete
71.2% Complete
73.1% Complete
75.0% Complete
76.9% Complete
78.8% Complete
80.8% Complete
82.7% Complete
84.6% Co

75.0% Complete
76.9% Complete
78.8% Complete
80.8% Complete
82.7% Complete
84.6% Complete
86.5% Complete
88.5% Complete
90.4% Complete
92.3% Complete
94.2% Complete
96.2% Complete
98.1% Complete
100.0% Complete
Results/Fuel1_Solar25_Wind25/fuel1_solar2.5_wind2.5_LowHome_HighWork_Timers9pm_noWPcontrol_penlevel0.5_storagebefore
Solving optimization.
1.9% Complete
3.8% Complete
5.8% Complete
7.7% Complete
9.6% Complete
11.5% Complete
13.5% Complete
15.4% Complete
17.3% Complete
19.2% Complete
21.2% Complete
23.1% Complete
25.0% Complete
26.9% Complete
28.8% Complete
30.8% Complete
32.7% Complete
34.6% Complete
36.5% Complete
38.5% Complete
40.4% Complete
42.3% Complete
44.2% Complete
46.2% Complete
48.1% Complete
50.0% Complete
51.9% Complete
53.8% Complete
55.8% Complete
57.7% Complete
59.6% Complete
61.5% Complete
63.5% Complete
65.4% Complete
67.3% Complete
69.2% Complete
71.2% Complete
73.1% Complete
75.0% Complete
76.9% Complete
78.8% Complete
80.8% Complete
82.7% Complete
84.6% Comp

71.2% Complete
73.1% Complete
75.0% Complete
76.9% Complete
78.8% Complete
80.8% Complete
82.7% Complete
84.6% Complete
86.5% Complete
88.5% Complete
90.4% Complete
92.3% Complete
94.2% Complete
96.2% Complete
98.1% Complete
100.0% Complete
Results/Fuel1_Solar25_Wind25/fuel1_solar2.5_wind2.5_LowHome_LowWork_TimersRandom_noWPcontrol_penlevel0.5
1.9% Complete
3.8% Complete
5.8% Complete
7.7% Complete
9.6% Complete
11.5% Complete
13.5% Complete
15.4% Complete
17.3% Complete
19.2% Complete
21.2% Complete
23.1% Complete
25.0% Complete
26.9% Complete
28.8% Complete
30.8% Complete
32.7% Complete
34.6% Complete
36.5% Complete
38.5% Complete
40.4% Complete
42.3% Complete
44.2% Complete
46.2% Complete
48.1% Complete
50.0% Complete
51.9% Complete
53.8% Complete
55.8% Complete
57.7% Complete
59.6% Complete
61.5% Complete
63.5% Complete
65.4% Complete
67.3% Complete
69.2% Complete
71.2% Complete
73.1% Complete
75.0% Complete
76.9% Complete
78.8% Complete
80.8% Complete
82.7% Complete
84.6% Complete

...

65.4% Complete
67.3% Complete
69.2% Complete
71.2% Complete
73.1% Complete
75.0% Complete
76.9% Complete
78.8% Complete
80.8% Complete
82.7% Complete
84.6% Complete
86.5% Complete
88.5% Complete
90.4% Complete
92.3% Complete
94.2% Complete
96.2% Complete
98.1% Complete
100.0% Complete
Solving optimization.
Storage Rate Result: 0
Storage Capacity:  0
Results/Fuel1_Solar25_Wind25/fuel1_solar2.5_wind2.5_HighHome_TimersRandom_noWPcontrol_penlevel1.0
----Capacity too low----
Try with storage:
1.9% Complete
3.8% Complete
5.8% Complete
7.7% Complete
9.6% Complete
11.5% Complete
13.5% Complete
15.4% Complete
17.3% Complete
19.2% Complete
21.2% Complete
23.1% Complete
25.0% Complete
26.9% Complete
28.8% Complete
30.8% Complete
32.7% Complete
34.6% Complete
36.5% Complete
38.5% Complete
40.4% Complete
42.3% Complete
44.2% Complete
46.2% Complete
48.1% Complete
50.0% Complete
51.9% Complete
53.8% Complete
55.8% Complete
57.7% Complete
59.6% Complete
61.5% Complete
63.5% Complete
65.4% Complete
67

88.5% Complete
90.4% Complete
92.3% Complete
94.2% Complete
96.2% Complete
98.1% Complete
100.0% Complete
Solving optimization.
Storage Rate Result: 0
Storage Capacity:  0
Results/Fuel1_Solar25_Wind25/fuel1_solar2.5_wind2.5_HighHome_TimersNone_WPcontrol_avgem_penlevel1.0
----Capacity too low----
Try with storage:
1.9% Complete
3.8% Complete
5.8% Complete
7.7% Complete
9.6% Complete
11.5% Complete
13.5% Complete
15.4% Complete
17.3% Complete
19.2% Complete
21.2% Complete
23.1% Complete
25.0% Complete
26.9% Complete
28.8% Complete
30.8% Complete
32.7% Complete
34.6% Complete
36.5% Complete
38.5% Complete
40.4% Complete
42.3% Complete
44.2% Complete
46.2% Complete
48.1% Complete
50.0% Complete
51.9% Complete
53.8% Complete
55.8% Complete
57.7% Complete
59.6% Complete
61.5% Complete
63.5% Complete
65.4% Complete
67.3% Complete
69.2% Complete
71.2% Complete
73.1% Complete
75.0% Complete
76.9% Complete
78.8% Complete
80.8% Complete
82.7% Complete
84.6% Complete
86.5% Complete
88.5% Complete


Storage Rate Result: 0
Storage Capacity:  0
Results/Fuel1_Solar25_Wind25/fuel1_solar2.5_wind2.5_LowHome_HighWork_Timers12am_noWPcontrol_penlevel1.0
----Capacity too low----
Try with storage:
1.9% Complete
3.8% Complete
5.8% Complete
7.7% Complete
9.6% Complete
11.5% Complete
13.5% Complete
15.4% Complete
17.3% Complete
19.2% Complete
21.2% Complete
23.1% Complete
25.0% Complete
26.9% Complete
28.8% Complete
30.8% Complete
32.7% Complete
34.6% Complete
36.5% Complete
38.5% Complete
40.4% Complete
42.3% Complete
44.2% Complete
46.2% Complete
48.1% Complete
50.0% Complete
51.9% Complete
53.8% Complete
55.8% Complete
57.7% Complete
59.6% Complete
61.5% Complete
63.5% Complete
65.4% Complete
67.3% Complete
69.2% Complete
71.2% Complete
73.1% Complete
75.0% Complete
76.9% Complete
78.8% Complete
80.8% Complete
82.7% Complete
84.6% Complete
86.5% Complete
88.5% Complete
90.4% Complete
92.3% Complete
94.2% Complete
96.2% Complete
98.1% Complete
100.0% Complete
Solving optimization.
Storage Rat

----Capacity too low----
Try with storage:
1.9% Complete
3.8% Complete
5.8% Complete
7.7% Complete
9.6% Complete
11.5% Complete
13.5% Complete
15.4% Complete
17.3% Complete
19.2% Complete
21.2% Complete
23.1% Complete
25.0% Complete
26.9% Complete
28.8% Complete
30.8% Complete
32.7% Complete
34.6% Complete
36.5% Complete
38.5% Complete
40.4% Complete
42.3% Complete
44.2% Complete
46.2% Complete
48.1% Complete
50.0% Complete
51.9% Complete
53.8% Complete
55.8% Complete
57.7% Complete
59.6% Complete
61.5% Complete
63.5% Complete
65.4% Complete
67.3% Complete
69.2% Complete
71.2% Complete
73.1% Complete
75.0% Complete
76.9% Complete
78.8% Complete
80.8% Complete
82.7% Complete
84.6% Complete
86.5% Complete
88.5% Complete
90.4% Complete
92.3% Complete
94.2% Complete
96.2% Complete
98.1% Complete
100.0% Complete
Solving optimization.
Storage Rate Result: 1676
Storage Capacity:  6705
Results/Fuel1_Solar25_Wind25/fuel1_solar2.5_wind2.5_LowHome_LowWork_TimersRandom_noWPcontrol_penlevel1.0_stor

13.5% Complete
15.4% Complete
17.3% Complete
19.2% Complete
21.2% Complete
23.1% Complete
25.0% Complete
26.9% Complete
28.8% Complete
30.8% Complete
32.7% Complete
34.6% Complete
36.5% Complete
38.5% Complete
40.4% Complete
42.3% Complete
44.2% Complete
46.2% Complete
48.1% Complete
50.0% Complete
51.9% Complete
53.8% Complete
55.8% Complete
57.7% Complete
59.6% Complete
61.5% Complete
63.5% Complete
65.4% Complete
67.3% Complete
69.2% Complete
71.2% Complete
73.1% Complete
75.0% Complete
76.9% Complete
78.8% Complete
80.8% Complete
82.7% Complete
84.6% Complete
86.5% Complete
88.5% Complete
90.4% Complete
92.3% Complete
94.2% Complete
96.2% Complete
98.1% Complete
100.0% Complete
Solving optimization.
Storage Rate Result: 3954
Storage Capacity:  15816
Results/Fuel1_Solar25_Wind25/fuel1_solar2.5_wind2.5_LowHome_LowWork_TimersNone_WPcontrol_avgem_penlevel1.0_storagebefore
Solving optimization.
----Capacity too low----
Try with storage:
1.9% Complete
3.8% Complete
5.8% Complete
7.7% Com

In [15]:
max_rate = 10000 # assumes average 4 hour storage 
cap = 40000

# Load capacity result so know if need to cover with storage
maxpen_results2 = pd.read_csv('Results/max_penetration_levels_1h_20220330.csv', index_col=0)
maxpen_results2_bau = pd.read_csv('Results/max_penetration_levels_1h_bau_20220330.csv', index_col=0)

timer_names = {'':'Timers9pm', '_midnighttimer':'Timers12am', '_NoTimers':'TimersNone', '_RandomTimers':'TimersRandom'}

fuel = 1
ev_scenario_date = '20220313'
ev_scenario = 'BusinessAsUsual'
ev_workplace_bool=True
ev_timers=''
wp_control = 'minpeak'
solar = 2.5
wind = 2.5

for penlevel in [0.5, 1.0]:

    # storage before
    grid = FutureGrid(gd_short)
    grid.set_up_scenario(year=2030, solar=solar, wind=wind, fuel=fuel, ev_scenario=ev_scenario, 
                                                ev_timers=ev_timers, ev_pen=penlevel, ev_workplace_control=wp_control, ev_workplace_bool=True, evs_bool=True, 
                                                ev_scenario_date=ev_scenario_date, weekend_date=ev_scenario_date,  weekend_timers=ev_timers)
    save_str = 'Results/Fuel1_Solar25_Wind25/fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_'+'TimersMixed'+'_WPcontrol_'+wp_control+'_penlevel'+str(penlevel)+'_storagebefore'
    str1 = 'fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_WPcontrol_'+wp_control
    print(save_str)
    grid.check_overgeneration(save_str=save_str, change_demand=False)
    grid.run_storage_before_capacitydispatch(cap, max_rate, allow_negative=True)
    grid.storage.df.to_csv(save_str+'_storagebeforedf_'+'20220330'+'.csv')
    grid.future.demand['demand'] = np.copy(grid.storage.df.comb_demand_after_storage.values)
    grid.check_overgeneration(save_str=save_str, change_demand=True, extra_save_str='_afterstorage')
    if penlevel < 0.8:
        max_penlevel = 1.0
    grid.run_dispatch(max_penlevel, save_str, result_date='20220330')  
                    
                    

Results/Fuel1_Solar25_Wind25/fuel1_solar2.5_wind2.5_BusinessAsUsual_TimersMixed_WPcontrol_minpeak_penlevel0.5_storagebefore
Solving optimization.
1.9% Complete
3.8% Complete
5.8% Complete
7.7% Complete
9.6% Complete
11.5% Complete
13.5% Complete
15.4% Complete
17.3% Complete
19.2% Complete
21.2% Complete
23.1% Complete
25.0% Complete
26.9% Complete
28.8% Complete
30.8% Complete
32.7% Complete
34.6% Complete
36.5% Complete
38.5% Complete
40.4% Complete
42.3% Complete
44.2% Complete
46.2% Complete
48.1% Complete
50.0% Complete
51.9% Complete
53.8% Complete
55.8% Complete
57.7% Complete
59.6% Complete
61.5% Complete
63.5% Complete
65.4% Complete
67.3% Complete
69.2% Complete
71.2% Complete
73.1% Complete
75.0% Complete
76.9% Complete
78.8% Complete
80.8% Complete
82.7% Complete
84.6% Complete
86.5% Complete
88.5% Complete
90.4% Complete
92.3% Complete
94.2% Complete
96.2% Complete
98.1% Complete
100.0% Complete
Results/Fuel1_Solar25_Wind25/fuel1_solar2.5_wind2.5_BusinessAsUsual_TimersMixe

# Other Renewables

# Dispatch alternative grid models

Wind and solar 1X or 5X 2019 levels

In [9]:
max_rate = 10000 # assumes average 4 hour storage 
cap = 40000

# Load capacity result so know if need to cover with storage
maxpen_results2 = pd.read_csv('Results/max_penetration_levels_1h_20220330.csv', index_col=0)
maxpen_results2_bau = pd.read_csv('Results/max_penetration_levels_1h_bau_20220330.csv', index_col=0)

timer_names = {'':'Timers9pm', '_midnighttimer':'Timers12am', '_NoTimers':'TimersNone', '_RandomTimers':'TimersRandom'}

fuel = 1
ev_scenario_date = '20220313'

save_folder_name = {5:'Fuel1_Solar5_Wind5', 1:'Fuel1_Solar1_Wind1'}

for penlevel in [0.5]:#, 1.0]:
    for solar, wind in {5:5}.items():
        for ev_scenario in ['UniversalHome', 'HighHome', 'LowHome_HighWork', 'LowHome_LowWork']: # Access scenarios
            # no workplace control
            ev_workplace_bool=False
            for i, ev_timers in enumerate(['_RandomTimers', '', '_midnighttimer', '_NoTimers']):
                # Case without storage before
#                 grid = FutureGrid(gd_short)
#                 grid.set_up_scenario(year=2030, solar=solar, wind=wind, fuel=fuel, ev_scenario=ev_scenario, ev_timers=ev_timers, ev_pen=penlevel, 
#                                      ev_workplace_control='', ev_workplace_bool=False, evs_bool=True, ev_scenario_date=ev_scenario_date, 
#                                      weekend_date=ev_scenario_date,  weekend_timers=ev_timers)
                save_str = 'Results/'+save_folder_name[solar]+'/fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_'+timer_names[ev_timers]+'_noWPcontrol'+'_penlevel'+str(penlevel)
                print(save_str)
                str1 = 'fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_noWPcontrol'
                max_penlevel = 1.0#maxpen_results2.loc[timer_names[ev_timers], str1]
#                 grid.check_overgeneration(save_str=save_str)
#                 grid.run_dispatch(max_penlevel, save_str, result_date='20220330') 

                # Case with 10GW storage before
                grid = FutureGrid(gd_short)
                grid.set_up_scenario(year=2030, solar=solar, wind=wind, fuel=fuel, ev_scenario=ev_scenario, 
                                     ev_timers=ev_timers, ev_pen=penlevel, ev_workplace_control='', ev_workplace_bool=False, evs_bool=True, 
                                     ev_scenario_date=ev_scenario_date, weekend_date=ev_scenario_date,  weekend_timers=ev_timers)
                save_str = 'Results/'+save_folder_name[solar]+'/fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_'+timer_names[ev_timers]+'_noWPcontrol'+'_penlevel'+str(penlevel)+'_storagebefore'
                print(save_str)
                str1 = 'fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_noWPcontrol'
                grid.check_overgeneration(save_str=save_str, change_demand=False)
                grid.run_storage_before_capacitydispatch(cap, max_rate, allow_negative=True)
                grid.storage.df.to_csv(save_str+'_storagebeforedf_'+'20220330'+'.csv')
                # Change demand dispatched to fossil fuel generators based on storage operations:
                grid.future.demand['demand'] = np.copy(grid.storage.df.comb_demand_after_storage.values)
                grid.check_overgeneration(save_str=save_str, change_demand=True, extra_save_str='_afterstorage')
                if penlevel < 0.8:
                    max_penlevel = 1.0
                grid.run_dispatch(max_penlevel, save_str, result_date='20220330') 

            # workplace control - repeat above without timers but with workplace controls
            ev_workplace_bool=True
            ev_timers = '_NoTimers'
            for wp_control in ['avgem', 'minpeak']:
                # Case without storage
#                 grid = FutureGrid(gd_short)
#                 grid.set_up_scenario(year=2030, solar=solar, wind=wind, fuel=fuel, ev_scenario=ev_scenario, 
#                                                         ev_timers=ev_timers, ev_pen=penlevel, ev_workplace_control=wp_control, ev_workplace_bool=True, evs_bool=True, 
#                                                         ev_scenario_date=ev_scenario_date, weekend_date=ev_scenario_date,  weekend_timers=ev_timers)
                save_str = 'Results/'+save_folder_name[solar]+'/fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_'+timer_names[ev_timers]+'_WPcontrol_'+wp_control+'_penlevel'+str(penlevel)
                print(save_str)
                str1 = 'fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_WPcontrol_'+wp_control
                max_penlevel = 1.0#maxpen_results2.loc[timer_names[ev_timers], str1]
#                 grid.check_overgeneration(save_str=save_str)
#                 grid.run_dispatch(max_penlevel, save_str, result_date='20220330')

                # Case with storage before
                grid = FutureGrid(gd_short)
                grid.set_up_scenario(year=2030, solar=solar, wind=wind, fuel=fuel, ev_scenario=ev_scenario, 
                                                        ev_timers=ev_timers, ev_pen=penlevel, ev_workplace_control=wp_control, ev_workplace_bool=True, evs_bool=True, 
                                                        ev_scenario_date=ev_scenario_date, weekend_date=ev_scenario_date,  weekend_timers=ev_timers)

                save_str = 'Results/'+save_folder_name[solar]+'/fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_'+timer_names[ev_timers]+'_WPcontrol_'+wp_control+'_penlevel'+str(penlevel)+'_storagebefore'
                str1 = 'fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_WPcontrol_'+wp_control
                print(save_str)
                grid.check_overgeneration(save_str=save_str, change_demand=False)
                grid.run_storage_before_capacitydispatch(cap, max_rate, allow_negative=True)
                grid.storage.df.to_csv(save_str+'_storagebeforedf_'+'20220330'+'.csv')
                grid.future.demand['demand'] = np.copy(grid.storage.df.comb_demand_after_storage.values)
                grid.check_overgeneration(save_str=save_str, change_demand=True, extra_save_str='_afterstorage')

                if penlevel < 0.8:
                    max_penlevel = 1.0
                grid.run_dispatch(max_penlevel, save_str, result_date='20220330') 
                
                

        ev_scenario = 'BusinessAsUsual'
        ev_workplace_bool=True
        ev_timers=''
        wp_control = 'minpeak'
#         grid = FutureGrid(gd_short)
#         grid.set_up_scenario(year=2030, solar=solar, wind=wind, fuel=fuel, ev_scenario=ev_scenario, 
#                                                     ev_timers=ev_timers, ev_pen=penlevel, ev_workplace_control=wp_control, ev_workplace_bool=True, evs_bool=True, 
#                                                     ev_scenario_date=ev_scenario_date, weekend_date=ev_scenario_date,  weekend_timers=ev_timers)
        save_str = 'Results/'+save_folder_name[solar]+'/fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_'+'TimersMixed'+'_WPcontrol_'+wp_control+'_penlevel'+str(penlevel)
        str1 = 'fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_WPcontrol_'+wp_control
        max_penlevel = 1.0#maxpen_results2_bau.loc['TimersMixed', str1]
        print(save_str)
#         grid.check_overgeneration(save_str=save_str)
#         grid.run_dispatch(max_penlevel, save_str, result_date='20220330')                
            
        # storage before
        grid = FutureGrid(gd_short)
        grid.set_up_scenario(year=2030, solar=solar, wind=wind, fuel=fuel, ev_scenario=ev_scenario, 
                                                    ev_timers=ev_timers, ev_pen=penlevel, ev_workplace_control=wp_control, ev_workplace_bool=True, evs_bool=True, 
                                                    ev_scenario_date=ev_scenario_date, weekend_date=ev_scenario_date,  weekend_timers=ev_timers)
        save_str = 'Results/'+save_folder_name[solar]+'/fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_'+'TimersMixed'+'_WPcontrol_'+wp_control+'_penlevel'+str(penlevel)+'_storagebefore'
        str1 = 'fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_WPcontrol_'+wp_control
        print(save_str)
        grid.check_overgeneration(save_str=save_str, change_demand=False)
        grid.run_storage_before_capacitydispatch(cap, max_rate)
        grid.storage.df.to_csv(save_str+'_storagebeforedf_'+'20220330'+'.csv')
        grid.future.demand['demand'] = np.copy(grid.storage.df.comb_demand_after_storage.values)
        grid.check_overgeneration(save_str=save_str, change_demand=True, extra_save_str='_afterstorage')
        if penlevel < 0.8:
            max_penlevel = 1.0
        grid.run_dispatch(max_penlevel, save_str, result_date='20220330')  
                    
                    

Results/Fuel1_Solar5_Wind5/fuel1_solar5_wind5_UniversalHome_TimersRandom_noWPcontrol_penlevel0.5
Results/Fuel1_Solar5_Wind5/fuel1_solar5_wind5_UniversalHome_TimersRandom_noWPcontrol_penlevel0.5_storagebefore
Solving optimization.
1.9% Complete
3.8% Complete
5.8% Complete
7.7% Complete
9.6% Complete
11.5% Complete
13.5% Complete
15.4% Complete
17.3% Complete
19.2% Complete
21.2% Complete
23.1% Complete
25.0% Complete
26.9% Complete
28.8% Complete
30.8% Complete
32.7% Complete
34.6% Complete
36.5% Complete
38.5% Complete
40.4% Complete
42.3% Complete
44.2% Complete
46.2% Complete
48.1% Complete
50.0% Complete
51.9% Complete
53.8% Complete
55.8% Complete
57.7% Complete
59.6% Complete
61.5% Complete
63.5% Complete
65.4% Complete
67.3% Complete
69.2% Complete
71.2% Complete
73.1% Complete
75.0% Complete
76.9% Complete
78.8% Complete
80.8% Complete
82.7% Complete
84.6% Complete
86.5% Complete
88.5% Complete
90.4% Complete
92.3% Complete
94.2% Complete
96.2% Complete
98.1% Complete
100.0% Com

Solving optimization.
1.9% Complete
3.8% Complete
5.8% Complete
7.7% Complete
9.6% Complete
11.5% Complete
13.5% Complete
15.4% Complete
17.3% Complete
19.2% Complete
21.2% Complete
23.1% Complete
25.0% Complete
26.9% Complete
28.8% Complete
30.8% Complete
32.7% Complete
34.6% Complete
36.5% Complete
38.5% Complete
40.4% Complete
42.3% Complete
44.2% Complete
46.2% Complete
48.1% Complete
50.0% Complete
51.9% Complete
53.8% Complete
55.8% Complete
57.7% Complete
59.6% Complete
61.5% Complete
63.5% Complete
65.4% Complete
67.3% Complete
69.2% Complete
71.2% Complete
73.1% Complete
75.0% Complete
76.9% Complete
78.8% Complete
80.8% Complete
82.7% Complete
84.6% Complete
86.5% Complete
88.5% Complete
90.4% Complete
92.3% Complete
94.2% Complete
96.2% Complete
98.1% Complete
100.0% Complete
Results/Fuel1_Solar5_Wind5/fuel1_solar5_wind5_HighHome_TimersNone_noWPcontrol_penlevel0.5
Results/Fuel1_Solar5_Wind5/fuel1_solar5_wind5_HighHome_TimersNone_noWPcontrol_penlevel0.5_storagebefore
Solving 

19.2% Complete
21.2% Complete
23.1% Complete
25.0% Complete
26.9% Complete
28.8% Complete
30.8% Complete
32.7% Complete
34.6% Complete
36.5% Complete
38.5% Complete
40.4% Complete
42.3% Complete
44.2% Complete
46.2% Complete
48.1% Complete
50.0% Complete
51.9% Complete
53.8% Complete
55.8% Complete
57.7% Complete
59.6% Complete
61.5% Complete
63.5% Complete
65.4% Complete
67.3% Complete
69.2% Complete
71.2% Complete
73.1% Complete
75.0% Complete
76.9% Complete
78.8% Complete
80.8% Complete
82.7% Complete
84.6% Complete
86.5% Complete
88.5% Complete
90.4% Complete
92.3% Complete
94.2% Complete
96.2% Complete
98.1% Complete
100.0% Complete
Results/Fuel1_Solar5_Wind5/fuel1_solar5_wind5_LowHome_HighWork_TimersNone_WPcontrol_minpeak_penlevel0.5
Results/Fuel1_Solar5_Wind5/fuel1_solar5_wind5_LowHome_HighWork_TimersNone_WPcontrol_minpeak_penlevel0.5_storagebefore
Solving optimization.
1.9% Complete
3.8% Complete
5.8% Complete
7.7% Complete
9.6% Complete
11.5% Complete
13.5% Complete
15.4% Comp

TypeError: '<=' not supported between instances of 'NoneType' and 'float'

In [14]:
max_rate = 10000 # assumes average 4 hour storage 
cap = 40000

# Load capacity result so know if need to cover with storage
# maxpen_results2 = pd.read_csv('Results/max_penetration_levels_1h_20220330.csv', index_col=0)
# maxpen_results2_bau = pd.read_csv('Results/max_penetration_levels_1h_bau_20220330.csv', index_col=0)

timer_names = {'':'Timers9pm', '_midnighttimer':'Timers12am', '_NoTimers':'TimersNone', '_RandomTimers':'TimersRandom'}

fuel = 1
ev_scenario_date = '20220313'

save_folder_name = {5:'Fuel1_Solar5_Wind5', 1:'Fuel1_Solar1_Wind1'}

penlevel = 0.5
solar = 5
wind = 5

ev_scenario = 'BusinessAsUsual'
ev_workplace_bool=True
ev_timers=''
wp_control = 'minpeak'
grid = FutureGrid(gd_short)
grid.set_up_scenario(year=2030, solar=solar, wind=wind, fuel=fuel, ev_scenario=ev_scenario, 
                                                    ev_timers=ev_timers, ev_pen=penlevel, ev_workplace_control=wp_control, ev_workplace_bool=True, evs_bool=True, 
                                                    ev_scenario_date=ev_scenario_date, weekend_date=ev_scenario_date,  weekend_timers=ev_timers)
save_str = 'Results/'+save_folder_name[solar]+'/fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_'+'TimersMixed'+'_WPcontrol_'+wp_control+'_penlevel'+str(penlevel)+'_storagebefore'
str1 = 'fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_WPcontrol_'+wp_control
print(save_str)
grid.check_overgeneration(save_str=save_str, change_demand=False)
grid.run_storage_before_capacitydispatch(cap, max_rate, allow_negative=True)
grid.storage.df.to_csv(save_str+'_storagebeforedf_'+'20220330'+'.csv')
grid.future.demand['demand'] = np.copy(grid.storage.df.comb_demand_after_storage.values)
grid.check_overgeneration(save_str=save_str, change_demand=True, extra_save_str='_afterstorage')
if penlevel < 0.8:
    max_penlevel = 1.0
grid.run_dispatch(max_penlevel, save_str, result_date='20220330')  

Results/Fuel1_Solar5_Wind5/fuel1_solar5_wind5_BusinessAsUsual_TimersMixed_WPcontrol_minpeak_penlevel0.5_storagebefore
Solving optimization.
1.9% Complete
3.8% Complete
5.8% Complete
7.7% Complete
9.6% Complete
11.5% Complete
13.5% Complete
15.4% Complete
17.3% Complete
19.2% Complete
21.2% Complete
23.1% Complete
25.0% Complete
26.9% Complete
28.8% Complete
30.8% Complete
32.7% Complete
34.6% Complete
36.5% Complete
38.5% Complete
40.4% Complete
42.3% Complete
44.2% Complete
46.2% Complete
48.1% Complete
50.0% Complete
51.9% Complete
53.8% Complete
55.8% Complete
57.7% Complete
59.6% Complete
61.5% Complete
63.5% Complete
65.4% Complete
67.3% Complete
69.2% Complete
71.2% Complete
73.1% Complete
75.0% Complete
76.9% Complete
78.8% Complete
80.8% Complete
82.7% Complete
84.6% Complete
86.5% Complete
88.5% Complete
90.4% Complete
92.3% Complete
94.2% Complete
96.2% Complete
98.1% Complete
100.0% Complete


In [ ]:
max_rate = 10000 # assumes average 4 hour storage 
cap = 40000

# Load capacity result so know if need to cover with storage
maxpen_results2 = pd.read_csv('Results/max_penetration_levels_1h_20220330.csv', index_col=0)
maxpen_results2_bau = pd.read_csv('Results/max_penetration_levels_1h_bau_20220330.csv', index_col=0)

timer_names = {'':'Timers9pm', '_midnighttimer':'Timers12am', '_NoTimers':'TimersNone', '_RandomTimers':'TimersRandom'}

fuel = 1
ev_scenario_date = '20220313'

save_folder_name = {5:'Fuel1_Solar5_Wind5', 1:'Fuel1_Solar1_Wind1'}

for penlevel in [0.5]:#, 1.0]:
    for solar, wind in {1:1}.items():
        for ev_scenario in ['UniversalHome', 'HighHome', 'LowHome_HighWork', 'LowHome_LowWork']: # Access scenarios
            # no workplace control
            ev_workplace_bool=False
            for i, ev_timers in enumerate(['_RandomTimers', '', '_midnighttimer', '_NoTimers']):
                # Case without storage before
#                 grid = FutureGrid(gd_short)
#                 grid.set_up_scenario(year=2030, solar=solar, wind=wind, fuel=fuel, ev_scenario=ev_scenario, ev_timers=ev_timers, ev_pen=penlevel, 
#                                      ev_workplace_control='', ev_workplace_bool=False, evs_bool=True, ev_scenario_date=ev_scenario_date, 
#                                      weekend_date=ev_scenario_date,  weekend_timers=ev_timers)
                save_str = 'Results/'+save_folder_name[solar]+'/fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_'+timer_names[ev_timers]+'_noWPcontrol'+'_penlevel'+str(penlevel)
                print(save_str)
                str1 = 'fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_noWPcontrol'
                max_penlevel = maxpen_results2.loc[timer_names[ev_timers], str1]
#                 grid.check_overgeneration(save_str=save_str)
#                 grid.run_dispatch(max_penlevel, save_str, result_date='20220330') 

                # Case with 10GW storage before
                grid = FutureGrid(gd_short)
                grid.set_up_scenario(year=2030, solar=solar, wind=wind, fuel=fuel, ev_scenario=ev_scenario, 
                                     ev_timers=ev_timers, ev_pen=penlevel, ev_workplace_control='', ev_workplace_bool=False, evs_bool=True, 
                                     ev_scenario_date=ev_scenario_date, weekend_date=ev_scenario_date,  weekend_timers=ev_timers)
                save_str = 'Results/'+save_folder_name[solar]+'/fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_'+timer_names[ev_timers]+'_noWPcontrol'+'_penlevel'+str(penlevel)+'_storagebefore'
                print(save_str)
                str1 = 'fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_noWPcontrol'
                grid.check_overgeneration(save_str=save_str, change_demand=False)
                grid.run_storage_before_capacitydispatch(cap, max_rate, allow_negative=True)
                grid.storage.df.to_csv(save_str+'_storagebeforedf_'+'20220330'+'.csv')
                # Change demand dispatched to fossil fuel generators based on storage operations:
                grid.future.demand['demand'] = np.copy(grid.storage.df.comb_demand_after_storage.values)
                grid.check_overgeneration(save_str=save_str, change_demand=True, extra_save_str='_afterstorage')
                if penlevel < 0.8:
                    max_penlevel = 1.0
                grid.run_dispatch(max_penlevel, save_str, result_date='20220330') 

            # workplace control - repeat above without timers but with workplace controls
            ev_workplace_bool=True
            ev_timers = '_NoTimers'
            for wp_control in ['avgem', 'minpeak']:
                # Case without storage
#                 grid = FutureGrid(gd_short)
#                 grid.set_up_scenario(year=2030, solar=solar, wind=wind, fuel=fuel, ev_scenario=ev_scenario, 
#                                                         ev_timers=ev_timers, ev_pen=penlevel, ev_workplace_control=wp_control, ev_workplace_bool=True, evs_bool=True, 
#                                                         ev_scenario_date=ev_scenario_date, weekend_date=ev_scenario_date,  weekend_timers=ev_timers)
                save_str = 'Results/'+save_folder_name[solar]+'/fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_'+timer_names[ev_timers]+'_WPcontrol_'+wp_control+'_penlevel'+str(penlevel)
                print(save_str)
                str1 = 'fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_WPcontrol_'+wp_control
                max_penlevel = maxpen_results2.loc[timer_names[ev_timers], str1]
#                 grid.check_overgeneration(save_str=save_str)
#                 grid.run_dispatch(max_penlevel, save_str, result_date='20220330')

                # Case with storage before
                grid = FutureGrid(gd_short)
                grid.set_up_scenario(year=2030, solar=solar, wind=wind, fuel=fuel, ev_scenario=ev_scenario, 
                                                        ev_timers=ev_timers, ev_pen=penlevel, ev_workplace_control=wp_control, ev_workplace_bool=True, evs_bool=True, 
                                                        ev_scenario_date=ev_scenario_date, weekend_date=ev_scenario_date,  weekend_timers=ev_timers)

                save_str = 'Results/'+save_folder_name[solar]+'/fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_'+timer_names[ev_timers]+'_WPcontrol_'+wp_control+'_penlevel'+str(penlevel)+'_storagebefore'
                str1 = 'fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_WPcontrol_'+wp_control
                print(save_str)
                grid.check_overgeneration(save_str=save_str, change_demand=False)
                grid.run_storage_before_capacitydispatch(cap, max_rate, allow_negative=True)
                grid.storage.df.to_csv(save_str+'_storagebeforedf_'+'20220330'+'.csv')
                grid.future.demand['demand'] = np.copy(grid.storage.df.comb_demand_after_storage.values)
                grid.check_overgeneration(save_str=save_str, change_demand=True, extra_save_str='_afterstorage')

                if penlevel < 0.8:
                    max_penlevel = 1.0
                grid.run_dispatch(max_penlevel, save_str, result_date='20220330') 
                
                

        ev_scenario = 'BusinessAsUsual'
        ev_workplace_bool=True
        ev_timers=''
        wp_control = 'minpeak'
#         grid = FutureGrid(gd_short)
#         grid.set_up_scenario(year=2030, solar=solar, wind=wind, fuel=fuel, ev_scenario=ev_scenario, 
#                                                     ev_timers=ev_timers, ev_pen=penlevel, ev_workplace_control=wp_control, ev_workplace_bool=True, evs_bool=True, 
#                                                     ev_scenario_date=ev_scenario_date, weekend_date=ev_scenario_date,  weekend_timers=ev_timers)
        save_str = 'Results/'+save_folder_name[solar]+'/fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_'+'TimersMixed'+'_WPcontrol_'+wp_control+'_penlevel'+str(penlevel)
        str1 = 'fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_WPcontrol_'+wp_control
        max_penlevel = maxpen_results2_bau.loc['TimersMixed', str1]
#         print(save_str)
#         grid.check_overgeneration(save_str=save_str)
#         grid.run_dispatch(max_penlevel, save_str, result_date='20220330')                
            
        # storage before
        grid = FutureGrid(gd_short)
        grid.set_up_scenario(year=2030, solar=solar, wind=wind, fuel=fuel, ev_scenario=ev_scenario, 
                                                    ev_timers=ev_timers, ev_pen=penlevel, ev_workplace_control=wp_control, ev_workplace_bool=True, evs_bool=True, 
                                                    ev_scenario_date=ev_scenario_date, weekend_date=ev_scenario_date,  weekend_timers=ev_timers)
        save_str = 'Results/'+save_folder_name[solar]+'/fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_'+'TimersMixed'+'_WPcontrol_'+wp_control+'_penlevel'+str(penlevel)+'_storagebefore'
        str1 = 'fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_WPcontrol_'+wp_control
        print(save_str)
        grid.check_overgeneration(save_str=save_str, change_demand=False)
        grid.run_storage_before_capacitydispatch(cap, max_rate)
        grid.storage.df.to_csv(save_str+'_storagebeforedf_'+'20220330'+'.csv')
        grid.future.demand['demand'] = np.copy(grid.storage.df.comb_demand_after_storage.values)
        grid.check_overgeneration(save_str=save_str, change_demand=True, extra_save_str='_afterstorage')
        if penlevel < 0.8:
            max_penlevel = 1.0
        grid.run_dispatch(max_penlevel, save_str, result_date='20220330')  
                    
                    